In [34]:
import plotly.express as px
import re
import plotly.io as pio
# pio.renderers.default = "browser"
pio.renderers.default = "notebook_connected"
from collections import OrderedDict
import argparse
from scipy.stats import gmean
import numpy as np
import plotly.graph_objects as go
import plotly.figure_factory as ff
import os
import sys
sys.path.append('..')
from util.spec2017_util import (
    bcolors,
    create_folder,
    yaml_load,
    yaml_overwrite,
)
from glob import glob
from scipy.stats import gmean
import copy


In [35]:
import pandas as pd
import numpy as np
pd.options.plotting.backend = "plotly"
if not os.path.exists("img"):
    os.mkdir("img")
w=3.3115
h=1.3

In [36]:
yml_file = "../util/spec2017_default.yml"
spec2017 = yaml_load(yml_file)
benchname = [dict_["name"] for dict_ in spec2017.values()]
print(benchname)
snapshot_out = "out_entropy/"

['perlbench', 'gcc', 'bwaves', 'mcf', 'cactuBSSN', 'namd', 'parest', 'povray', 'lbm', 'omnetpp', 'wrf', 'xalancbmk', 'x264', 'blender', 'cam4', 'deepsjeng', 'imagick', 'leela', 'nab', 'exchange2', 'fotonik3d', 'roms', 'xz']


search all SPEC2017 benchmark to find what to show in plots

In [37]:
bench_arr = []
bench_dict = dict()
for name in benchname:
    # print(name)
    files = glob(os.path.join(snapshot_out, f"{name}.*"))
    numf = len(files)
    # print(f"spec {name} contains {numf} outputs.")
    if numf == 0: continue
    # 2 comp schemes, 3 xor scheme, 2 constraints, 2 inter intra
    if (numf == 3): 
        bench_arr.append(name)

In [38]:
gem5_dir = "/home/zhewen/repo/gem5-dev/gem5"
num_ss = []
num_sample = 10
bench_arr_anno = []
bench_num_arr = []
bench_cat_arr = []
for name,num,val in zip(benchname, spec2017.keys(), spec2017.values()):
    if name in bench_arr:
        ckpt = gem5_dir + f"/ckpt/spec2017/1c-2GB-valgrind/{num}.{name}"
        num_ckpt = len(glob(os.path.join(ckpt, "cpt.*", "")))
        print(f"spec {num}.{name} contains {num_ckpt} ckpts.")
        num_ss.append(num_ckpt*num_sample)
        bench_arr_anno.append(f"{name} ({num_ckpt*num_sample})")
        bench_num_arr.append(num)
        # print(val)
        bench_cat_arr.append(val['category'])
print(num_ss)
print(bench_cat_arr)

spec 500.perlbench contains 14 ckpts.
spec 503.bwaves contains 19 ckpts.
spec 505.mcf contains 22 ckpts.
spec 507.cactuBSSN contains 23 ckpts.
spec 508.namd contains 26 ckpts.
spec 510.parest contains 22 ckpts.
spec 511.povray contains 20 ckpts.
spec 519.lbm contains 24 ckpts.
spec 523.xalancbmk contains 17 ckpts.
spec 526.blender contains 15 ckpts.
spec 531.deepsjeng contains 19 ckpts.
spec 538.imagick contains 4 ckpts.
spec 541.leela contains 19 ckpts.
spec 544.nab contains 15 ckpts.
spec 548.exchange2 contains 15 ckpts.
spec 549.fotonik3d contains 23 ckpts.
spec 554.roms contains 25 ckpts.
[140, 190, 220, 230, 260, 220, 200, 240, 170, 150, 190, 40, 190, 150, 150, 230, 250]
['int', 'float', 'int', 'float', 'float', 'float', 'float', 'float', 'int', 'float', 'int', 'float', 'int', 'float', 'int', 'float', 'float']


initialize overall dataframe (dict_panda) and stat dataframe (bench_df)

In [39]:
df = pd.DataFrame(
    index=bench_arr, 
    data={
        'ss_theoretical':num_ss, 
        'bench_num':bench_num_arr,
        'category': bench_cat_arr,
    }
)
display(df)

,ss_theoretical,bench_num,category
perlbench,140,500,int
bwaves,190,503,float
mcf,220,505,int
cactuBSSN,230,507,float
namd,260,508,float
parest,220,510,float
povray,200,511,float
lbm,240,519,float
xalancbmk,170,523,int
blender,150,526,float


 construct dataframe and fill

In [40]:
xor_schemes = ["none", "rand", "ideal"]
df_line_ari = copy.deepcopy(df)
df_line_geo = copy.deepcopy(df)
df_bit_ari = copy.deepcopy(df)
df_bit_geo = copy.deepcopy(df)
df_ss_actual = copy.deepcopy(df)

for xor in xor_schemes:
    line_ari = []
    line_geo = []
    bit_ari = []
    bit_geo = []
    ss_arr = []
    for name,ss in zip(bench_arr, num_ss):
        f = open(snapshot_out + f"{name}.entropy_{xor}", "r")
        str_ = f.readline()
        if len(str_.split(','))==5:
            line_ari_ = float(str_.split(',')[0])
            bit_ari_ = float(str_.split(',')[2])
            ss_ = float(str_.split(',')[4])
            line_geo_ = float(str_.split(',')[1])
            bit_geo_ = float(str_.split(',')[3])
            line_ari.append(line_ari_)
            ss_arr.append(ss_)
            line_geo.append(line_geo_)
            bit_ari.append(bit_ari_)
            bit_geo.append(bit_geo_)
        else: assert False
        f.close()
    
    # fill dataframe
    df_line_ari[f'{xor}'] = line_ari
    df_line_geo[f'{xor}'] = line_geo
    df_bit_ari[f'{xor}'] = bit_ari
    df_bit_geo[f'{xor}'] = bit_geo
    df_ss_actual[f'{xor}'] = ss_arr

df_num = df.select_dtypes(include='number')
gm_arr = gmean(df_num.iloc[:,:],axis=0)
df.loc['geomean'] = np.NaN
df.loc['geomean',df_num.columns.tolist()] = gm_arr


summary of all data

In [41]:
display(df_line_geo)

,ss_theoretical,bench_num,category,none,rand,ideal
perlbench,140,500,int,10.843351,9.998498,9.844388
bwaves,190,503,float,9.785185,9.013472,8.992798
mcf,220,505,int,10.988947,9.999681,9.903834
cactuBSSN,230,507,float,10.623277,9.944663,9.641362
namd,260,508,float,10.945496,5.499844,5.444049
parest,220,510,float,8.583361,5.035397,4.264198
povray,200,511,float,10.804228,5.453446,5.415187
lbm,240,519,float,10.828590,5.489527,5.402449
xalancbmk,170,523,int,10.840222,5.497547,5.378956
blender,150,526,float,10.935782,5.495289,5.474047


In [42]:
df_line_geo_drop = df_line_geo.drop(['ss_theoretical','bench_num', 'category'],axis=1)
df_bit_geo_drop = df_bit_geo.drop(['ss_theoretical','bench_num', 'category'],axis=1)
display(df_line_geo_drop)

,none,rand,ideal
perlbench,10.843351,9.998498,9.844388
bwaves,9.785185,9.013472,8.992798
mcf,10.988947,9.999681,9.903834
cactuBSSN,10.623277,9.944663,9.641362
namd,10.945496,5.499844,5.444049
parest,8.583361,5.035397,4.264198
povray,10.804228,5.453446,5.415187
lbm,10.828590,5.489527,5.402449
xalancbmk,10.840222,5.497547,5.378956
blender,10.935782,5.495289,5.474047


## plot entropy

In [43]:
from plotly.subplots import make_subplots
fig = make_subplots(rows=2, cols=1)
fig.update_layout(
    # template='simple_white',
    title=f"XOR boost snapshot profiling (geomean)\
        <br><sup>entropy</sup>",
        yaxis_title="entropy",
        xaxis_title=None,
    font=dict(
        family="Courier New, monospace",
        size=18,
    ),
    legend=dict(
        orientation="h",
        itemwidth=30,
        yanchor="bottom",
        y=1.02,
        xanchor="center",
        x=0.5,
    ),
    barmode='group',
)
# fig.update_yaxes(range = [0,4])

i = 0
colors = [px.colors.sequential.Blugrn,px.colors.sequential.Purpor]
hatch = ['', '-']
marker = ['o', 'x']

fig.add_trace(
    go.Bar(
        x=df_bit_geo_drop.index,
        y=df_bit_geo_drop.none,
        name='bit none',
        marker_color=colors[0][0]
    ),
    row=1,col=1,
)
fig.add_trace(
    go.Bar(
        x=df_bit_geo_drop.index,
        y=df_bit_geo_drop.rand,
        name='bit rand',
        marker_color=colors[0][1]
    ),
    row=1,col=1,
)
fig.add_trace(
    go.Bar(
        x=df_bit_geo_drop.index,
        y=df_bit_geo_drop.ideal,
        name='bit ideal',
        marker_color=colors[0][2]
    ),
    row=1,col=1,
)
fig.add_trace(
    go.Bar(
        x=df_line_geo_drop.index,
        y=df_line_geo_drop.none,
        name='line none',
        marker_color=colors[1][3]
    ),
    row=2,col=1,
)
fig.add_trace(
    go.Bar(
        x=df_line_geo_drop.index,
        y=df_line_geo_drop.rand,
        name='line rand',
        marker_color=colors[1][4]
    ),
    row=2,col=1,
)
fig.add_trace(
    go.Bar(
        x=df_line_geo_drop.index,
        y=df_line_geo_drop.ideal,
        name='line ideal',
        marker_color=colors[1][5]
    ),
    row=2,col=1,
)

fig.show()

# write static image
dpi = 300
fig.update_traces(marker_line_width = 0,
                  selector=dict(type="bar"),
                  marker_pattern_shape=None,)

fig.update_layout(
    template='plotly_white',
    width=2*w*dpi,# double col
    height=h*dpi,
    paper_bgcolor='rgba(0,0,0,0)',
    title=None,
    xaxis_title=None,
    yaxis_title="entropy",
    margin=dict(l=10, r=10, t=10, b=10),
    font=dict(
        family="Courier New, monospace",
        size=25,
    ),
    bargap=0.10,
    bargroupgap=0.0,
)

fig.update_xaxes(tickangle=-45,
    type='category',
    # tickvals=np.arange(len(new)),
    # ticktext=new,
    linewidth=0,
)
fig.update_yaxes(
    gridwidth=0
)
# fig.for_each_trace(lambda t: t.update(marker_))

fig.write_image("img/entropy_geo.pdf", 
                width=2*w*dpi+50,
                height=h*dpi+50)


check all snapshots profiled

In [44]:
fig_ss = go.Figure()
fig_ss.update_layout(
    title=f"XOR boost snapshot profiling\
        <br><sup>entropy</sup>",
        yaxis_title="# snapshots",
        xaxis_title=None,
    font=dict(
        family="Courier New, monospace",
        size=18,
    )
)
i = 0
colors = [px.colors.sequential.Purp,px.colors.sequential.Tealgrn]
for xor in df.xor.unique():
    q_overall = df.loc[(df['xor']==xor)]
    q = q_overall['bench'].values[0]
    color = colors[int(q_overall.xor=='none')][i]

    fig_ss.add_trace(
        go.Bar(
            x=q.index, 
            y=q.ss_actual, 
            name=f'{xor}', 
            opacity=1,
            marker_color=color,
            marker_pattern_shape="x",
            error_y=dict(
                type='data',
                array=q.ss_theoretical-q.ss_actual,
                arrayminus=q.ss_theoretical - q.ss_theoretical,
                visible=True,
            )
        ),
    )

    i+=1
fig_ss.show()


AttributeError: 'DataFrame' object has no attribute 'xor'